In [3]:
import FinanceDataReader as fdr

In [4]:
from datetime import datetime, timedelta

def get_stock_history(symbol, duration=30):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    df = fdr.DataReader(symbol, start_date, end_date)
    return df.to_csv()

df = get_stock_history('AAPL', 30)
df


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-02-07,190.639999,191.050003,188.610001,189.410004,189.168625,53439000
2024-02-08,189.389999,189.539993,187.350006,188.320007,188.080017,40962000
2024-02-09,188.649994,189.990005,188.000000,188.850006,188.850006,45155200
2024-02-12,188.419998,188.669998,186.789993,187.149994,187.149994,41781900
2024-02-13,185.770004,186.210007,183.509995,185.039993,185.039993,56529500
2024-02-14,185.320007,185.529999,182.440002,184.149994,184.149994,54630500
2024-02-15,183.550003,184.490005,181.350006,183.860001,183.860001,65434500
2024-02-16,183.419998,184.850006,181.669998,182.309998,182.309998,49701400
2024-02-20,181.789993,182.429993,180.000000,181.559998,181.559998,53665600


In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import tempfile
import os
from fastapi.responses import FileResponse
from IPython.display import Image, display

# 차트 생성
def get_stock_history(symbol, duration=30):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=duration)
    df = fdr.DataReader(symbol, start_date, end_date)
    # return df.to_csv()
    
    # 차트 생성
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['Close'], label='Close')
    plt.title(f'{symbol} Close Price Chart')    

    # 임시 파일로 차트 저장
    with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmpfile:
        plt.savefig(tmpfile.name)
        plt.close()  # 리소스 해제

    # 차트 이미지 파일 반환
    return FileResponse(path=tmpfile.name, filename=f"{symbol}_chart.png", media_type='image/png')

chart = get_stock_history('AAPL')
# display(Image(chart.file))



AttributeError: 'FileResponse' object has no attribute 'file'

In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
symbol = 'AAPL'
df = fdr.DataReader(symbol, start_date, end_date)


# # 차트 생성
# plt.figure(figsize=(10, 5))
# plt.plot(df.index, df['Close'], label='Close')
# plt.title(f'{symbol} Close Price Chart')

# # 임시 파일로 차트 저장
# with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmpfile:
#     plt.savefig(tmpfile.name)
#     plt.close()  # 리소스 해제

# # 차트 이미지 파일 반환
# FileResponse(path=tmpfile.name, filename=f"{symbol}_chart.png", media_type='image/png')


In [20]:
import matplotlib.ticker as ticker
from mpl_finance import candlestick2_ohlc

end_date = datetime.now()
start_date = end_date - timedelta(days=30)
symbol = 'AAPL'
df = fdr.DataReader(symbol, start_date, end_date)

df['MA3'] = df['Close'].rolling(window=3).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA10'] = df['Close'].rolling(window=10).mean()
df['MA20'] = df['Close'].rolling(window=20).mean()

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
index = df.index.astype('str') # 캔들스틱 x축이 str로 들어감

# 이동평균선 그리기
ax.plot(index, df['MA3'], label='MA3', linewidth=0.7)
ax.plot(index, df['MA5'], label='MA5', linewidth=0.7)
ax.plot(index, df['MA10'], label='MA10', linewidth=0.7)

# X축 티커 숫자 20개로 제한
ax.xaxis.set_major_locator(ticker.MaxNLocator(10))

# 그래프 title과 축 이름 지정
ax.set_title(f'{symbol} chart', fontsize=22)
ax.set_xlabel('Date')

# 캔들차트 그리기
candlestick2_ohlc(ax, df['Open'], df['High'], 
                  df['Low'], df['Close'],
                  width=0.5, colorup='r', colordown='b')
ax.legend()
plt.grid()
plt.show()

3974.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'mpl_finance'